In [12]:
%pylab
import pandas as pd
import pickle
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [17]:
path = os.path.expanduser('~/Desktop/')
fname1 = os.path.join(path, 'probs_started-2018-06-23-05-20-51-Nseen6045.pkl')
fname2 = os.path.join(path, 'probs_started-2018-06-23-16-53-08-Nseen6045.pkl')
with open(fname1, 'rb') as fh:
    probs1 = pickle.load(fh)
with open(fname2, 'rb') as fh:
    probs2 = pickle.load(fh)

In [31]:
print(np.argwhere(np.argmax(probs1, 1) != np.argmax(probs2, 1)).shape)

pred2 = pd.read_csv(os.path.join(path, 'pred_started-2018-06-23-16-53-08-Nseen6045.csv'))
pred1 = pd.read_csv(os.path.join(path, 'pred_started-2018-06-23-05-20-51-Nseen6045.csv'))

print(np.argwhere(pred2[pred2.columns[0]] != pred1[pred1.columns[0]]))

(45, 2)
[[  637]
 [  638]
 [ 1255]
 [ 1468]
 [ 1787]
 [ 1796]
 [ 1885]
 [ 2002]
 [ 2148]
 [ 2177]
 [ 2193]
 [ 3054]
 [ 3208]
 [ 3217]
 [ 3224]
 [ 3227]
 [ 3278]
 [ 3287]
 [ 3418]
 [ 3644]
 [ 3671]
 [ 3676]
 [ 3677]
 [ 4072]
 [ 4082]
 [ 4095]
 [ 4152]
 [ 4153]
 [ 4158]
 [ 4159]
 [ 4166]
 [ 4168]
 [ 4169]
 [ 4197]
 [ 4269]
 [ 4271]
 [ 4273]
 [ 4277]
 [ 4278]
 [ 4281]
 [ 4282]
 [ 4283]
 [ 4285]
 [ 4320]
 [ 4390]
 [ 4445]
 [ 4613]
 [ 4680]
 [ 4752]
 [ 5057]
 [ 5577]
 [ 5578]
 [ 5913]
 [ 6336]
 [ 6494]
 [ 6505]
 [ 6630]
 [ 6726]
 [ 6900]
 [ 6901]
 [ 7096]
 [ 7234]
 [ 7253]
 [ 8741]
 [ 8912]
 [ 8913]
 [ 9058]
 [ 9731]
 [10782]
 [10783]
 [10784]
 [11904]
 [11910]
 [11979]
 [12054]
 [12067]
 [12068]
 [12161]
 [12162]
 [12197]
 [12866]
 [13680]
 [13805]
 [13861]
 [13862]
 [13959]
 [13996]
 [14356]
 [14357]
 [14504]
 [14584]
 [15119]
 [15128]
 [15236]
 [15319]
 [15342]
 [15529]
 [15591]
 [15726]
 [16048]
 [16850]
 [16851]
 [17038]
 [17066]
 [17099]
 [17162]
 [17189]
 [17208]
 [17306]
 [17307]
 [